In [1]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import cv2
from IPython.display import display
import math, random
import time, datetime, sys, os, shutil
import operator

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import pandas as pd

#tf.config.optimizer.set_jit(True)
#tf.debugging.set_log_device_placement(True)
print(tf.config.experimental.list_physical_devices())

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons(SELECTED_TAXONS)
id_map_inv = get_selected_taxons(SELECTED_TAXONS, inv=True)
n_classes = len(list(id_map.keys()))

#tf.test.is_gpu_available()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Handle data

## Preparing panda arrays

In [2]:
X_train, y_train, _ = get_dataset(TRAIN_DATASET_PATH, ids=False)
X_test, y_test, _ = get_dataset(TEST_DATASET_PATH, ids=False)

print("Test length:", len(X_train), "- n classes:", len(np.unique(y_train)))
print("Test length:", len(X_test), "- n classes:", len(np.unique(y_test)))

# Balance dataset
train_dict = {}
for file, label in zip(X_train, y_train):
    train_dict.setdefault(label, []).append(file)
max_samples = np.max([len(train_dict[taxon_id]) for taxon_id in train_dict])
X_train = []
y_train = []
for taxon_id in train_dict:
    ratio = np.ceil(max_samples/len(train_dict[taxon_id]))
    tmp = np.repeat(train_dict[taxon_id], ratio)
    np.random.shuffle(tmp)
    train_dict[taxon_id] = tmp[0:max_samples]
    X_train.extend(tmp[0:max_samples])
    y_train.extend([taxon_id]*max_samples)

'187/187'

'187/187'

Test length: 7728 - n classes: 187
Test length: 1933 - n classes: 187


In [3]:
data_train = {'png_path':  X_train, 'taxon_id': y_train}
data_test = {'png_path':  X_test, 'taxon_id': y_test}

df_train = pd.DataFrame(data_train, columns = ['png_path', 'taxon_id'])
df_test = pd.DataFrame(data_test, columns = ['png_path', 'taxon_id'])
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Prtining some infos
print(len(df_train), len(df_test))
df_train.head()
df_test.head()

44880 1933


,png_path,taxon_id
0,/mnt/nvme-storage/pfauregi/datasets/condensed/...,ADMI
1,/mnt/nvme-storage/pfauregi/datasets/condensed/...,NROS
2,/mnt/nvme-storage/pfauregi/datasets/condensed/...,UULN
3,/mnt/nvme-storage/pfauregi/datasets/condensed/...,EOCO
4,/mnt/nvme-storage/pfauregi/datasets/condensed/...,CPLA


## Setting up the flows

In [4]:
datagen = ImageDataGenerator(rescale=1./255.,
                         rotation_range=90, 
                         brightness_range=[0.8,1.2], 
                         horizontal_flip=True, 
                         vertical_flip=True,
                         fill_mode='nearest',
                         width_shift_range=40,
                         height_shift_range=40,
                         zoom_range=0.2,
                         validation_split=0.2,
                         #featurewise_std_normalization=True,
                         data_format="channels_last")

#classes_array = np.unique(y_train)
train_generator = datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        #classes=classes_array,
        class_mode='categorical')
val_generator = datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=train_generator.class_indices,
        class_mode='categorical')

train_spe = train_generator.samples // BATCH_SIZE
val_spe = val_generator.samples // BATCH_SIZE

Found 44880 validated image filenames belonging to 187 classes.
Found 1933 validated image filenames belonging to 187 classes.


In [5]:
class_indices = train_generator.class_indices
f = open(os.path.join(SAVED_MODELS_ROOT, 'model_id_map.csv'), 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(["taxon", "id"])
    for taxon in class_indices:
        writer.writerow([taxon, class_indices[taxon]])

## Testing

In [ ]:
i = 0
stop = False
for batch in train_generator:
    images = batch[0]
    labels = batch[1]
    for i in range(images.shape[0]):
        print(np.argmax(labels[i]))
        image = (images[i,:,:,:]*255).astype('uint8')
        #print(image)
        display(Image.fromarray(image))
        i+=1
        if i>=1:
            stop = True
            break
    if stop: break;
    #display()

# Model desgin

In [6]:
# fetching base model
#base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None)
input_tensor = Input(shape=(256, 256, 3))
base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)

In [7]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(len(train_generator.class_indices.keys()), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

In [8]:
delete_all_files_in_folder(LOG_DIR)
#delete_all_files_in_folder(SAVED_MODELS_ROOT)

## Setting callbacks

In [9]:
# Setting tensorboard
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
log_file=os.path.join(SAVED_MODELS_ROOT, "model.log")
os.remove(log_file)
csv_logger = CSVLogger(log_file)

## New layers only

In [11]:
n_epochs = 3
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

3 epochs composed of 1402 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1402 steps, validate for 60 steps
Epoch 1/3
1402/1402 [==============================] - 560s 399ms/step - loss: 2.5353 - accuracy: 0.3848 - val_loss: 4.8561 - val_accuracy: 0.1125
Epoch 2/3
1402/1402 [==============================] - 557s 397ms/step - loss: 1.4669 - accuracy: 0.5837 - val_loss: 5.0873 - val_accuracy: 0.1146
Epoch 3/3
1402/1402 [==============================] - 557s 397ms/step - loss: 1.2738 - accuracy: 0.6296 - val_loss: 6.0196 - val_accuracy: 0.0943


## Training 2 last inceptions blocks

In [12]:
n_epochs = 7
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False,
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

7 epochs composed of 1402 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1402 steps, validate for 60 steps
Epoch 3/9
1402/1402 [==============================] - 562s 401ms/step - loss: 0.9469 - accuracy: 0.7101 - val_loss: 6.0124 - val_accuracy: 0.2047
Epoch 4/9
1402/1402 [==============================] - 558s 398ms/step - loss: 0.5575 - accuracy: 0.8175 - val_loss: 4.2887 - val_accuracy: 0.2901
Epoch 5/9
1402/1402 [==============================] - 559s 399ms/step - loss: 0.4435 - accuracy: 0.8515 - val_loss: 5.1655 - val_accuracy: 0.2396
Epoch 6/9
1402/1402 [==============================] - 558s 398ms/step - loss: 0.3715 - accuracy: 0.8748 - val_loss: 4.1834 - val_accuracy: 0.2932
Epoch 7/9
1402/1402 [==============================] - 558s 398ms/step - loss: 0.3301 - accuracy: 0.8896 - val_loss: 4.6402 - val_accuracy: 0.2859
Epoch 8/9
1402/1402 [==============================] - 558s 398ms/step - loss: 0.2947 - accuracy: 0.9032 - val_loss

## Training full model

## Saving model

In [ ]:
n_epochs = 24
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

24 epochs composed of 1402 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1402 steps, validate for 60 steps
Epoch 9/32
1402/1402 [==============================] - 561s 400ms/step - loss: 0.7972 - accuracy: 0.7558 - val_loss: 2.0869 - val_accuracy: 0.5318
Epoch 10/32
1402/1402 [==============================] - 557s 397ms/step - loss: 0.4155 - accuracy: 0.8642 - val_loss: 0.8294 - val_accuracy: 0.7516
Epoch 11/32
1402/1402 [==============================] - 556s 397ms/step - loss: 0.3266 - accuracy: 0.8915 - val_loss: 0.8736 - val_accuracy: 0.7490
Epoch 12/32
1402/1402 [==============================] - 557s 397ms/step - loss: 0.2776 - accuracy: 0.9062 - val_loss: 0.6159 - val_accuracy: 0.8198
Epoch 13/32
1402/1402 [==============================] - 556s 396ms/step - loss: 0.2397 - accuracy: 0.9196 - val_loss: 1.2052 - val_accuracy: 0.7161
Epoch 14/32
1402/1402 [==============================] - 556s 396ms/step - loss: 0.2100 - accuracy: 0.927

In [ ]:
save_model(model, SAVED_MODELS_ROOT)